<a href="https://colab.research.google.com/github/protoi/Speech-Emotion-Analysis/blob/main/1_Extracting_Audio_File_Paths.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DESCRIPTION

### READING THE DATASETS FROM KAGGLE, EXTRACTING THEIR PATHS AND PUTTING IT IN A CSV TO BE USED IN Audio Processing and Silence Removal.ipynb

# LIBRARIES AND IMPORTS

In [ ]:
!pip install pydub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pathlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math as math
import librosa as lib
import librosa.display as lid
import re 
from pydub import AudioSegment
from pydub.silence import split_on_silence
from pydub.playback import play
import os as os
import glob as glob
from pathlib import Path
from os import chdir
from scipy.signal import butter, filtfilt
import gc
from tqdm import tqdm
from google.colab import drive
import shutil

***LINUX UTILITY COMMANDS***

In [ ]:
# shutil.rmtree( "path of folder to delete" , ignore_errors=True)

# DATA COLLECTION AND TABLE MAKING

## KAGGLE

**you need to mount google drive before you run the other commands**

In [ ]:
!pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
!cp /content/drive/MyDrive/Machine\ Learning/Speech\ Emotion\ Recognition/kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

## SAVEE

_______________________________


***SAVEE***

***LOADING THE DATASET***

In [ ]:
! kaggle datasets download barelydedicated/savee-database

savee-database.zip: Skipping, found more recently modified local copy (use --force to force download)


***UNZIPPING FILES AND REMOVING A REDUNDANT FOLDER***

In [ ]:
!unzip savee-database.zip -d "/content/SAVEE"
shutil.rmtree( "/content/SAVEE/audiodata" , ignore_errors=True)

Archive:  savee-database.zip
replace /content/SAVEE/AudioData/DC/a01.wav? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
  inflating: /content/SAVEE/audiodata/AudioData/DC/a01.wav  
  inflating: /content/SAVEE/audiodata/AudioData/DC/a02.wav  
  inflating: /content/SAVEE/audiodata/AudioData/DC/a03.wav  
  inflating: /content/SAVEE/audiodata/AudioData/DC/a04.wav  
  inflating: /content/SAVEE/audiodata/AudioData/DC/a05.wav  
  inflating: /content/SAVEE/audiodata/AudioData/DC/a06.wav  
  inflating: /content/SAVEE/audiodata/AudioData/DC/a07.wav  
  inflating: /content/SAVEE/audiodata/AudioData/DC/a08.wav  
  inflating: /content/SAVEE/audiodata/AudioData/DC/a09.wav  
  inflating: /content/SAVEE/audiodata/AudioData/DC/a10.wav  
  inflating: /content/SAVEE/audiodata/AudioData/DC/a11.wav  
  inflating: /content/SAVEE/audiodata/AudioData/DC/a12.wav  
  inflating: /content/SAVEE/audiodata/AudioData/DC/a13.wav  
  inflating: /content/SAVEE/audiodata/AudioData/DC/a14.wav  
  inflating: /content/SAVEE/audi

***LISTING ALL .wav FILES RECURSIVELY***

In [ ]:
SAVEE_FILE=glob.glob("/content/SAVEE/AudioData" + '/**/*.wav', recursive=True)
SAVEE_FILE.sort()

***MAKING THE SAVEE_DATAFRAME***

In [ ]:
filename=[]
filepath=[]
emotion=[]

SAVEE_dataset = {
    'filepath': [],
    'filename': [],
    'emotion': []

}
SAVEE_PATTERN = re.compile("([a-z]+)[0-9]+")
SAVEE_NAME_PATTERN = re.compile("([A-Z]{2}\/[a-z]+[0-9]+)")

for index, path in enumerate(SAVEE_FILE):
  filepath.append(path)
  patt = SAVEE_PATTERN.findall(path)[0]
  name = SAVEE_NAME_PATTERN.findall(path)[0].replace('/', '_')
  filename.append(name)
  emo=" "
  if patt == 'a':
    emo = 'anger'
  elif patt == 'd':
    emo = 'disgust'
  elif patt == 'f':
    emo = 'fear'
  elif patt == 'h':
    emo = 'happy'
  elif patt == 'n': 
    emo = 'neutral'
  elif patt == 'sa': 
    emo = 'sadness'
  elif patt == 'su':
    emo = 'surprise'

  emotion.append('male_' + emo)

SAVEE_DATAFRAME = pd.DataFrame(emotion, columns=['labels'])
SAVEE_DATAFRAME['source'] = "SAVEE"
SAVEE_DATAFRAME = pd.concat([SAVEE_DATAFRAME, pd.DataFrame(filepath, columns=['path'])], axis=1)
SAVEE_DATAFRAME = pd.concat([SAVEE_DATAFRAME, pd.DataFrame(filename, columns=['name'])], axis=1)

SAVEE_DATAFRAME.labels.value_counts()
# SAVEE_DATAFRAME

male_neutral     120
male_anger        60
male_disgust      60
male_fear         60
male_happy        60
male_sadness      60
male_surprise     60
Name: labels, dtype: int64

## RAVDESS

_____________
***RAVDESS***

***LOADING THE DATASET***

In [ ]:
! kaggle datasets download uwrfkaggler/ravdess-emotional-speech-audio

ravdess-emotional-speech-audio.zip: Skipping, found more recently modified local copy (use --force to force download)


***UNZIPPING THE DATASET***

In [ ]:
!unzip ravdess-emotional-speech-audio.zip -d "/content/RAVDESS"
shutil.rmtree( "/content/RAVDESS/audio_speech_actors_01-24" , ignore_errors=True)

Archive:  ravdess-emotional-speech-audio.zip
replace /content/RAVDESS/Actor_01/03-01-01-01-01-01-01.wav? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
  inflating: /content/RAVDESS/audio_speech_actors_01-24/Actor_01/03-01-01-01-01-01-01.wav  
  inflating: /content/RAVDESS/audio_speech_actors_01-24/Actor_01/03-01-01-01-01-02-01.wav  
  inflating: /content/RAVDESS/audio_speech_actors_01-24/Actor_01/03-01-01-01-02-01-01.wav  
  inflating: /content/RAVDESS/audio_speech_actors_01-24/Actor_01/03-01-01-01-02-02-01.wav  
  inflating: /content/RAVDESS/audio_speech_actors_01-24/Actor_01/03-01-02-01-01-01-01.wav  
  inflating: /content/RAVDESS/audio_speech_actors_01-24/Actor_01/03-01-02-01-01-02-01.wav  
  inflating: /content/RAVDESS/audio_speech_actors_01-24/Actor_01/03-01-02-01-02-01-01.wav  
  inflating: /content/RAVDESS/audio_speech_actors_01-24/Actor_01/03-01-02-01-02-02-01.wav  
  inflating: /content/RAVDESS/audio_speech_actors_01-24/Actor_01/03-01-02-02-01-01-01.wav  
  inflating: /content/RAVDE

***LISTING ALL THE .wav FILES INSIDE THE RAVDESS DIRECTORY RECURSIVELY***

In [ ]:
RAVDESS_FILE=glob.glob("/content/RAVDESS/" + '/**/*.wav', recursive=True)
RAVDESS_FILE.sort()

***MAKING THE RAVDESS DATAFRAME***

In [ ]:
filename=[]
filepath=[]
emotion=[]
RAVDESS_PATTERN = re.compile("(\d\d-\d\d-\d\d-\d\d-\d\d-\d\d-\d\d)")
RAVDESS_NAME_PATTERN = re.compile("(Actor_\d\d\/\d\d-\d\d-\d\d-\d\d-\d\d-\d\d-\d\d)")


# "\d\d-\d\d-\d\d-\d\d-\d\d-\d\d-\d\d"
for index, path in enumerate(RAVDESS_FILE):
  filepath.append(path)
  patt = RAVDESS_PATTERN.findall(path)[0]
  if(path[4] == "2"):
    print(f'{patt} was dropped')
    continue
  name = RAVDESS_NAME_PATTERN.findall(path)[0].replace('/', '_')
  filename.append(name)
  gender = "male_"

  if int(name[7]) % 2 == 0:
    gender = "female_"
  
  temp = int(patt[7])
  emo = None
  if temp == 1:
    emo = 'neutral'
  elif temp == 2:
    emo = 'calm'
  elif temp == 3:
    emo = 'happy'
  elif temp == 4:
    emo = 'sadness'
  elif temp == 5: 
    emo = 'anger'
  elif temp == 6: 
    emo = 'fear'
  elif temp == 7:
    emo = 'disgust'
  elif temp == 8:
    emo = 'surprise'
  else:
    print(path)

  emotion.append(gender + emo)

RAVDESS_DATAFRAME = pd.DataFrame(emotion, columns=['labels'])
RAVDESS_DATAFRAME['source'] = "RAVDESS"
RAVDESS_DATAFRAME = pd.concat([RAVDESS_DATAFRAME, pd.DataFrame(filepath, columns=['path'])], axis=1)
RAVDESS_DATAFRAME = pd.concat([RAVDESS_DATAFRAME, pd.DataFrame(filename, columns=['name'])], axis=1)

RAVDESS_DATAFRAME.labels.value_counts()

male_calm          96
male_happy         96
male_sadness       96
male_anger         96
male_fear          96
male_disgust       96
male_surprise      96
female_calm        96
female_happy       96
female_sadness     96
female_anger       96
female_fear        96
female_disgust     96
female_surprise    96
male_neutral       48
female_neutral     48
Name: labels, dtype: int64

## TESS

_______________________________


***TESS***

***LOADING THE DATASET***

In [ ]:
! kaggle datasets download ejlok1/toronto-emotional-speech-set-tess

toronto-emotional-speech-set-tess.zip: Skipping, found more recently modified local copy (use --force to force download)


***UNZIPPING THE DATASET***

In [ ]:
!unzip toronto-emotional-speech-set-tess.zip -d "/content/TESS"
shutil.rmtree( "/content/TESS/tess toronto emotional speech set data" , ignore_errors=True)
# shutil.rmtree( "/content/TESS/tess\ toronto\ emotional\ speech\ set\ data" , ignore_errors=True)

Archive:  toronto-emotional-speech-set-tess.zip
replace /content/TESS/TESS Toronto emotional speech set data/OAF_Fear/OAF_back_fear.wav? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
  inflating: /content/TESS/tess toronto emotional speech set data/TESS Toronto emotional speech set data/OAF_Fear/OAF_back_fear.wav  
  inflating: /content/TESS/tess toronto emotional speech set data/TESS Toronto emotional speech set data/OAF_Fear/OAF_bar_fear.wav  
  inflating: /content/TESS/tess toronto emotional speech set data/TESS Toronto emotional speech set data/OAF_Fear/OAF_base_fear.wav  
  inflating: /content/TESS/tess toronto emotional speech set data/TESS Toronto emotional speech set data/OAF_Fear/OAF_bath_fear.wav  
  inflating: /content/TESS/tess toronto emotional speech set data/TESS Toronto emotional speech set data/OAF_Fear/OAF_bean_fear.wav  
  inflating: /content/TESS/tess toronto emotional speech set data/TESS Toronto emotional speech set data/OAF_Fear/OAF_beg_fear.wav  
  inflating: /content

***LISTING ALL THE .wav FILES INSIDE THE TESS DIRECTORY RECURSIVELY***

In [ ]:
TESS_FILE=glob.glob("/content/TESS/" + '/**/*.wav', recursive=True)
TESS_FILE.sort()

***MAKING THE TESS DATAFRAME***

In [ ]:
filename=[]
filepath=[]
emotion=[]

TESS_PATTERN = re.compile("[OY]AF_([a-zA-Z_]*)\/")
TESS_NAME_PATTERN = re.compile("([a-zA-Z\_]*).wav")

for index, path in enumerate(TESS_FILE):
  filepath.append(path)
  patt = TESS_PATTERN.findall(path)[0]
  name = TESS_NAME_PATTERN.findall(path)[0]
  filename.append(name)
  gender = "female_"
  emo = None

  if patt == 'disgust' or patt == 'Disgust':
    emo = 'disgust'
  elif patt == 'fear' or patt == 'Fear':
    emo = 'fear'
  elif patt == 'Pleasant_surprise' or patt == 'pleasant_surprised':
    emo = 'surprise'
  elif patt == 'Sad' or patt == 'sad':
    emo = 'sadness'
  elif patt == 'angry':
    emo = 'anger'
  else:
    emo = patt

  emotion.append(gender + emo)

TESS_DATAFRAME = pd.DataFrame(emotion, columns=['labels'])
TESS_DATAFRAME['source'] = "TESS"
TESS_DATAFRAME = pd.concat([TESS_DATAFRAME, pd.DataFrame(filepath, columns=['path'])], axis=1)
TESS_DATAFRAME = pd.concat([TESS_DATAFRAME, pd.DataFrame(filename, columns=['name'])], axis=1)

TESS_DATAFRAME.labels.value_counts()

female_fear        400
female_surprise    400
female_sadness     400
female_anger       400
female_disgust     400
female_happy       400
female_neutral     400
Name: labels, dtype: int64

## CREMA-D

_______________________________


***TESS***

***LOADING THE DATASET***

In [ ]:
! kaggle datasets download ejlok1/cremad

cremad.zip: Skipping, found more recently modified local copy (use --force to force download)


***UNZIPPING THE DATASET***

In [ ]:
!unzip cremad.zip -d "/content/CREMAD"

Archive:  cremad.zip
replace /content/CREMAD/AudioWAV/1001_DFA_ANG_XX.wav? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


***LISTING ALL THE .wav FILES INSIDE THE TESS DIRECTORY RECURSIVELY***

In [ ]:
CREMAD_FILE=glob.glob("/content/CREMAD" + '/**/*.wav', recursive=True)
CREMAD_FILE.sort()

***MAKING THE CREMA-D DATAFRAME***

In [ ]:
filename=[]
filepath=[]
emotion=[]
getFemale = [1002,1003,1004,1006,1007,1008,1009,1010,1012,1013,1018,1020,1021,1024,1025,1028,1029,1030,1037,1043,1046,1047,1049,
          1052,1053,1054,1055,1056,1058,1060,1061,1063,1072,1073,1074,1075,1076,1078,1079,1082,1084,1089,1091]
          
CREMAD_PATTERN = re.compile("([0-9]{4})")
CREMAD_NAME_PATTERN = re.compile("([a-zA-Z0-9_]*).wav")

for index, path in enumerate(CREMAD_FILE):
  filepath.append(path)
  patt = CREMAD_PATTERN.findall(path)[0]
  name = CREMAD_NAME_PATTERN.findall(path)[0]
  
  filename.append(name)
  gender = "male_"
  if int(patt) in getFemale:
    gender="female_" 
  

  temp = name[9:12]




  emo = None

  if temp == 'SAD':
    emo = 'sadness'
  elif temp == 'ANG':
    emo = 'anger'
  elif temp == 'DIS':
    emo = 'disgust'
  elif temp == 'FEA':
    emo = 'fear'
  elif temp == 'HAP':
    emo = 'happy'
  elif temp == 'NEU':
    emo = 'neutral'
  emotion.append(gender + emo)

CREMAD_DATAFRAME = pd.DataFrame(emotion, columns=['labels'])
CREMAD_DATAFRAME['source'] = "CREMAD"
CREMAD_DATAFRAME = pd.concat([CREMAD_DATAFRAME, pd.DataFrame(filepath, columns=['path'])], axis=1)
CREMAD_DATAFRAME = pd.concat([CREMAD_DATAFRAME, pd.DataFrame(filename, columns=['name'])], axis=1)

CREMAD_DATAFRAME.labels.value_counts()

male_anger        671
male_disgust      671
male_fear         671
male_happy        671
male_sadness      671
female_anger      600
female_disgust    600
female_fear       600
female_happy      600
female_sadness    600
male_neutral      575
female_neutral    512
Name: labels, dtype: int64

# DATAFRAME JOINING

## JOINING SAVEE, RAVDESS, TESS and CREMA-D

In [ ]:
df = pd.concat([SAVEE_DATAFRAME, RAVDESS_DATAFRAME, TESS_DATAFRAME, CREMAD_DATAFRAME], axis = 0)
# print(df.labels.value_counts())
# df.head()
df.to_csv("path_list.csv", index=False)

In [ ]:
len(df)

12162